## Download Actively Learned Model

In [1]:
! wget https://cloud.tsinghua.edu.cn/f/7d8f9e9b0f9e4c90b24b/?dl=1 -O training_data.xyz
! wget https://cloud.tsinghua.edu.cn/f/ae658d0e817942e9b1e7/?dl=1 -O m3gnet_HaberBosch.pth

--2024-02-20 17:50:15--  https://cloud.tsinghua.edu.cn/f/7d8f9e9b0f9e4c90b24b/?dl=1
Resolving cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)... 166.111.6.101, 2402:f000:1:406:166:111:6:101
Connecting to cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)|166.111.6.101|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cloud.tsinghua.edu.cn/seafhttp/files/1c2511b0-0e07-40b5-b541-4cce5fe4d000/total.xyz [following]
--2024-02-20 17:50:15--  https://cloud.tsinghua.edu.cn/seafhttp/files/1c2511b0-0e07-40b5-b541-4cce5fe4d000/total.xyz
Reusing existing connection to cloud.tsinghua.edu.cn:443.
HTTP request sent, awaiting response... 200 OK
Length: 289057855 (276M) [application/octet-stream]
Saving to: ‘training_data.xyz’

training_data.xyz   100%[===================>] 275.67M  30.6MB/s    in 9.0s    

2024-02-20 17:50:24 (30.6 MB/s) - ‘training_data.xyz’ saved [289057855/289057855]

--2024-02-20 17:50:25--  https://cloud.tsinghua.edu.cn/f/ae658d0e817942e9b1e7/?dl=1
R

## Absorption Energy Estimated by DFT and MLFF

In [1]:
! wget https://cloud.tsinghua.edu.cn/f/f469295aef2645af8fc2/?dl=1 -O H_aimd.xyz
! wget https://cloud.tsinghua.edu.cn/f/e5ba57bcc4fa4bb4a5c3/?dl=1 -O H2_aimd.xyz
! wget https://cloud.tsinghua.edu.cn/f/f9b9747cc195400d985c/?dl=1 -O N_aimd.xyz
! wget https://cloud.tsinghua.edu.cn/f/07d80f26ce9948af9196/?dl=1 -O N2_aimd.xyz
! wget https://cloud.tsinghua.edu.cn/f/a7a8bf835a544c7d8b8a/?dl=1 -O NH_aimd.xyz
! wget https://cloud.tsinghua.edu.cn/f/60646ccd7b0244f38100/?dl=1 -O NH2_aimd.xyz
! wget https://cloud.tsinghua.edu.cn/f/8fa5f7a799d14467a695/?dl=1 -O NH3_aimd.xyz
! wget https://cloud.tsinghua.edu.cn/f/fdc0d4a1fb72474c9051/?dl=1 -O slab_H_aimd.xyz
! wget https://cloud.tsinghua.edu.cn/f/e388b39787384da8881a/?dl=1 -O slab_H2_aimd.xyz
! wget https://cloud.tsinghua.edu.cn/f/28066154805b4d3f8699/?dl=1 -O slab_N_aimd.xyz
! wget https://cloud.tsinghua.edu.cn/f/1d77c91549054e76830a/?dl=1 -O slab_N2_aimd.xyz
! wget https://cloud.tsinghua.edu.cn/f/34396d43da2a4573a4da/?dl=1 -O slab_NH_aimd.xyz
! wget https://cloud.tsinghua.edu.cn/f/dc2d9bbc4b5545e6be00/?dl=1 -O slab_NH2_aimd.xyz
! wget https://cloud.tsinghua.edu.cn/f/df6950f22091404b923d/?dl=1 -O slab_NH3_aimd.xyz
! wget https://cloud.tsinghua.edu.cn/f/4a89220fd933410f85ab/?dl=1 -O slab_Ir_aimd.xyz

--2024-02-20 19:28:36--  https://cloud.tsinghua.edu.cn/f/f469295aef2645af8fc2/?dl=1
Resolving cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)... 166.111.6.101, 2402:f000:1:406:166:111:6:101
Connecting to cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)|166.111.6.101|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cloud.tsinghua.edu.cn/seafhttp/files/7c214dd5-5ec3-47b2-8818-be0c441ac78c/H_aimd_test.xyz [following]
--2024-02-20 19:28:36--  https://cloud.tsinghua.edu.cn/seafhttp/files/7c214dd5-5ec3-47b2-8818-be0c441ac78c/H_aimd_test.xyz
Reusing existing connection to cloud.tsinghua.edu.cn:443.
HTTP request sent, awaiting response... 200 OK
Length: 5100600 (4.9M) [application/octet-stream]
Saving to: ‘H_aimd.xyz’

H_aimd.xyz          100%[===================>]   4.86M  12.8MB/s    in 0.4s    

2024-02-20 19:28:37 (12.8 MB/s) - ‘H_aimd.xyz’ saved [5100600/5100600]

--2024-02-20 19:28:38--  https://cloud.tsinghua.edu.cn/f/e5ba57bcc4fa4bb4a5c3/?dl=1
Resolving

In [2]:
import os
from ase.io import read, write
import numpy as np
from kinetics.forcefield.potential import Potential, DeepCalculator
from kinetics.datasets.utils.build import build_dataloader
from kinetics.forcefield.potential import Potential, batch_to_dict
import torch

mole_list = ["N","N2","H","H2","NH","NH2","NH3"]
e_absorb_list = []
e_absorb_pred_list = []

def inference(dataloader, potential):
    predicted_energies=[]
    predicted_forces=[]
    predicted_stresses=[]
    for _, graph_batch in enumerate(dataloader):
        graph_batch = graph_batch.to(potential.device)
        input = batch_to_dict(graph_batch)
        results = potential(input, include_forces=True, include_stresses=True)
        energies = results["energies"]
        predicted_energies.extend(energies.detach().cpu().numpy().tolist())
        forces_tuple = torch.split(results["forces"].cpu().detach(), graph_batch.num_atoms.cpu().tolist(), dim=0)
        for force in forces_tuple:
            predicted_forces.append(np.array(force))
        predicted_stresses.extend(list(results["stresses"].cpu().detach().numpy()))
    return predicted_energies, predicted_forces, predicted_stresses

for i, mole in enumerate(mole_list):
    slab_atoms_list = read("slab_{}_aimd.xyz".format(mole), index=":")
    slab_energies = [atoms.get_potential_energy() for atoms in slab_atoms_list]
    slab_ir_atoms_list = read("slab_Ir_aimd.xyz", index=":")
    slab_ir_energies = [atoms.get_potential_energy() for atoms in slab_ir_atoms_list]
    mole_atoms_list = read("{}_aimd.xyz".format(mole), index=":")
    mole_energies = [atoms.get_potential_energy() for atoms in mole_atoms_list]
    e_absorb = np.mean(slab_energies) - np.mean(mole_energies) - np.mean(slab_ir_energies)
    e_absorb_list.append(e_absorb)
        
potential = Potential.load(load_path="m3gnet_HaberBosch.pth")
for i, mole in enumerate(mole_list):
    slab_atoms_list = read("./slab_{}_aimd.xyz".format(mole), index=":")
    slab_ir_atoms_list = read("./slab_Ir_aimd.xyz", index=":")
    mole_atoms_list = read("./{}_aimd.xyz".format(mole), index=":")
    slab_dataloader = build_dataloader(
        atoms = slab_atoms_list,
        only_inference=True,
        cutoff = 5.0,
        threebody_cutoff = 4.0,
        batch_size = 32,
        shuffle = False,
    )
    slab_ir_dataloader = build_dataloader(
        atoms = slab_ir_atoms_list,
        only_inference=True,
        cutoff=5.0,
        threebody_cutoff=4.0,
        batch_size=32,
        shuffle=False,
    )
    mole_dataloader = build_dataloader(
        atoms = mole_atoms_list,
        only_inference=True,
        cutoff=5.0,
        threebody_cutoff=4.0,
        batch_size=32,
        shuffle=False,
    )
    slab_energies, _, _ = inference(slab_dataloader, potential)
    slab_ir_energies, _, _ = inference(slab_ir_dataloader, potential)
    mole_energies, _, _ = inference(mole_dataloader, potential)
    e_absorb = np.mean(slab_energies) - np.mean(mole_energies) - np.mean(slab_ir_energies)
    e_absorb_pred_list.append(e_absorb)

        
for i, mole in enumerate(mole_list):
    print("{}: e_absorb: {:.4f}, e_absorb_pred: {:.4f}, mae: {}".format(mole, e_absorb_list[i], e_absorb_pred_list[i], abs(e_absorb_list[i]-e_absorb_pred_list[i])))

Loading the model from m3gnet_HaberBosch.pth
N: e_absorb: -9.6225, e_absorb_pred: -9.6194, mae: 0.003091011173694369
N2: e_absorb: -1.7470, e_absorb_pred: -1.7621, mae: 0.015115562322307596
H: e_absorb: -2.9918, e_absorb_pred: -2.9949, mae: 0.003111443085344945
H2: e_absorb: -1.3889, e_absorb_pred: -1.4548, mae: 0.06587901922699757
NH: e_absorb: -6.8027, e_absorb_pred: -6.8928, mae: 0.0900137484128436
NH2: e_absorb: -5.8862, e_absorb_pred: -5.9138, mae: 0.027590421338970827
NH3: e_absorb: -2.7908, e_absorb_pred: -2.6980, mae: 0.09286045025601197


## Example of Crashed MD without Active Learning

Below is the MD trajectory of a crashed Haber Bosch simulation. We conducted this offline MD simulation under 5400K with a M3GNET model trained on ~10k Ir-N-H structures. 

In [ ]:
! wget https://cloud.tsinghua.edu.cn/f/1e8a1be5934d43e18a2d/?dl=1 -O IrNH_5400K_crashed.xyz

We further conducted Haber Bosch simulation with the same temperature and initial structure. But this time, the M3GNET model was actively updated by our IDEAL algorithm during the simulation. It turned out that this time the simulation keeps stable and the trajectory is shown below:

In [ ]:
! wget https://cloud.tsinghua.edu.cn/f/c4207029d995472697a5/?dl=1 -O IrNH2_5400K_nocrashed.xyz